In [ ]:
import numpy as np
import pylab as pl
%matplotlib inline

# Introduction

In this notebook we'll run the same quantum circuit $H |0>$ using 5 different quantum programming packages:

- [Project Q](#Project-Q)
- [Qiskit](#Qiskit)
- [Quil](#Quil)
- [Cirq](#Cirq)
- [Q#](#Q$\#$)

## $H |0>$ on the Bloch sphere

In [ ]:
from qutip import Bloch, basis
from qutip import snot  # Hadamard gate

In [ ]:
snot()

In [ ]:
snot() * basis(2, 0)

In [ ]:
fig = pl.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')

b = Bloch(fig=fig, axes=ax)
state = snot() * basis(2, 0)
b.add_states(state)
b.render(fig=fig, axes=ax)

# Project Q

In [ ]:
from projectq import MainEngine
from projectq.ops import H, Measure

eng = MainEngine()
qubit = eng.allocate_qubit()

H | qubit
Measure | qubit

eng.flush()
print("Measured {}".format(int(qubit)))

# Qiskit

In [ ]:
from qiskit import BasicAer
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute

In [ ]:
# set up Quantum Register and Classical Register for 1 qubit
q = QuantumRegister(1)
c = ClassicalRegister(1)
qc = QuantumCircuit(q, c)
qc.h(q)
qc.measure(q, c)

In [ ]:
qc.draw(output='mpl')

In [ ]:
job = execute(qc, backend=BasicAer.get_backend('qasm_simulator'), shots=1000)
result = job.result()
result

In [ ]:
result.get_counts(qc)

Get quantum wavefunction

In [ ]:
job = execute(qc, backend=BasicAer.get_backend('statevector_simulator'))
result = job.result()
state = result.get_statevector()
state

# Quil

In [ ]:
from pyquil.api import QVMConnection
from pyquil.quil import Program, Declare
from pyquil.gates import X, MEASURE, RX, H
import subprocess

# Start the Lisp server in a subprocess
qvm_server = subprocess.Popen(["/src/qvm/qvm", "-S"])

# Connect to the QVM
qvm = QVMConnection()

In [ ]:
program = Program()
ro = program.declare('ro', 'BIT')
program += H(0)

In [ ]:
result = qvm.run(program + MEASURE(0, ro[0]), trials=1000)
result

In [ ]:
np.array(result).T[0].sum()

In [ ]:
wf = qvm.wavefunction(program)
wf.amplitudes

Stop the Lisp server

In [ ]:
qvm_server.terminate()

# Cirq

In [ ]:
import cirq
simulator = cirq.Simulator()

qubit = cirq.GridQubit(0, 0)
circuit = cirq.Circuit.from_ops(
    cirq.H(qubit),
    cirq.measure(qubit, key='m')
)

In [ ]:
print(circuit)

In [ ]:
result = simulator.run(circuit, repetitions=1000)

In [ ]:
result

In [ ]:
result.measurements["m"]

In [ ]:
np.array(result.measurements["m"]).T.sum()

# Q$\#$

Make sure to change the kernel to Q# before running this part.

In [ ]:
open Microsoft.Quantum.Extensions.Diagnostics; // import the DumpMachine function

operation Hadamard(count: Int) : ( Result[], Int )
    {
        mutable r = Zero;
        mutable numOnes = 0;
        mutable results = new Result[0];

        using (qubits = Qubit[1]) {
            for (test in 1..count)
            {
                H(qubits[0]);

                if (test == 1)
                {
                    DumpMachine(); // Show the current waveform
                }

                set r = M (qubits[0]);

                // Add to the results
                set results = results + [r];

                // Count the number of ones we saw:
                if (r == One)
                {
                    set numOnes = numOnes + 1;
                }

                ResetAll(qubits);
            }
        }

        return (results, numOnes);
    }

In [ ]:
operation HadamardRuns() : ( Result[], Int ) {
    let count = 10;
    return Hadamard(count);
}

In [ ]:
%simulate HadamardRuns